In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

from collections import Counter
from typing import List
from decimal import Decimal
import heapq
from tqdm import trange

import sympy
import numpy as np
import scipy.io as sio
import pysindy as ps
from derivative import dxdt
import sys; sys.path.insert(0, "../")
from best_subset import ps_features, brute_force
from sklearn.model_selection import train_test_split

Sklearn's version: 1.5.2


In [2]:
### Burgers ###
data = sio.loadmat('../Datasets/burgers.mat')
u = (data['usol']).real
x = (data['x'][0]).real
t = (data['t'][:,0]).real
dt = t[1]-t[0]
dx = x[2]-x[1]
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T

In [3]:
### GA (Loop of crossover -> mutation -> evaluate_genome) ###

def generate_module(n_derivatives):
    pde_module = sorted(np.random.randint(0, 3) for i in range(np.random.randint(1, n_derivatives+1)))
    return tuple(pde_module)

def generate_genome(n_modules, n_derivatives):
    genome = fset(generate_module(n_derivatives) for _ in range(n_modules))
    return genome

def generate_chromosome(n_modules, n_derivatives, pop_size):
    chromosome = set()
    count = 0
    while count < pop_size:
        genome = generate_genome(n_modules, n_derivatives)
        if genome not in chromosome:
            chromosome.add(genome)
            count += 1
    return chromosome

def display_derivative(n):
    nx = 'x'*n
    return sympy.symbols(f'u_{nx}')

def display_module(pde_module):
    return np.prod([display_derivative(_) for _ in pde_module])

# coefficients for sorted(encoded_pde)
def display_pde(encoded_pde, coefficients=None):
    if coefficients is None:
        coefficients = [1 for _ in range(len(encoded_pde))]
    out = []
    for module, c in zip(encoded_pde, coefficients):
        out.append(c*display_module(module))
    return sum(out)

# not efficient: O(len(genome))
def crossover(genome1: List, genome2: List, fs=True):
    if genome1 != genome2:
        while True: 
            idx1 = np.random.randint(len(genome1))
            idx2 = np.random.randint(len(genome2))
            if genome1[idx1] != genome2[idx2]:
                break
        genome1[idx1], genome2[idx2] = genome2[idx2], genome1[idx1]
    if fs: 
        genome1, genome2 = fset(genome1), fset(genome2)
    return genome1, genome2

# in-place func
def mutation(genome: List, n_derivatives: int, mutate_rate=0.4, fs=True):
    # add
    if np.random.uniform(0, 1) < mutate_rate:
        genome.append(generate_module(n_derivatives))
    # del
    if np.random.uniform(0, 1) < mutate_rate:
        lg = len(genome)
        if lg > 0:
            idx = np.random.randint(lg)
            genome.pop(idx)
    # order
    if np.random.uniform(0, 1) < mutate_rate:
        lg = len(genome)
        if lg > 0: 
            i = np.random.randint(lg)
            lg = len(genome[i])
            if lg > 0:
                genome[i] = list(genome[i])
                j = np.random.randint(len(genome[i]))
                if genome[i][j] == 0:
                    genome[i][j] = np.random.randint(1, n_derivatives)
                else:
                    genome[i][j] -= 1
                genome[i] = tuple(sorted(genome[i]))
    if fs: 
        genome = fset(genome)
    return genome

def numericalize_module(module, base_features):
    return np.prod([base_features[derivative] for derivative in module], axis=0)

def numericalize_genome(genome, base_features):
    return np.stack([numericalize_module(module, base_features) 
                     for module in genome], axis=-1)

def compute_genome_coefficient(genome, base_features, target):
    features = numericalize_genome(genome, base_features)
    assert target.shape == features[:, :, 0].shape
    n_features = features.shape[-1]
    target = target.reshape(-1, 1)
    features = features.reshape(-1, n_features)
    coeff, error, _, _ = np.linalg.lstsq(features, target, rcond=None)
    return coeff, error[0]

# Computing coefficients + fitness
def evaluate_genome(genome, base_features, target, epsilon=0):
    coeff, mse = compute_genome_coefficient(genome, base_features, target)
    mse = mse / np.prod(target.shape)
    fitness = mse + abs(epsilon)*len(genome)
    return fitness, coeff

### Miscellaneous for GA ###
def sci_format(n):
    sf = '%.2E' % Decimal(n)
    sf = sf.split('E')
    return float(sf[0]), int(sf[1])

def fset(ls): return frozenset(ls)

In [4]:
n_derivatives = 3
n_modules = 3
pop_size = 400
epsilons = [10**(i-6) for i in range(1, 6)] # 1e-5 to 1e-1

differentiation_method = ps.FiniteDifference(is_uniform=True)
u_t = differentiation_method._differentiate(u.T, t).T
u_x = differentiation_method._differentiate(u, x)
u_xx = differentiation_method._differentiate(u_x, x)
base_features = np.array([u, u_x, u_xx])

In [5]:
genome = generate_genome(n_modules, n_derivatives); print(genome)
print(evaluate_genome(genome, base_features, u_t))
display_pde(genome, [3,4,5])

frozenset({(0, 1), (0,), (0, 0, 2)})
(0.0003582229279224092, array([[-0.99037148],
       [-0.00265959],
       [ 0.17265538]]))


5*u_**2*u_xx + 3*u_*u_x + 4*u_

In [6]:
### testing cross over ###
# genome2 = generate_genome(n_modules, n_derivatives); print(genome2)
# genome, genome2 = crossover(list(genome), list(genome2))
# genome, genome2 = fset(genome), fset(genome2)
# print(genome, genome2)

## Cross-over-only GA ##

In [7]:
# n_generations = 10

# print("Initialization...")
# chrom = generate_chromosome(n_modules, n_derivatives, pop_size)
# set_chrom = chrom.copy()
# chrom = list(chrom)

# fitnesses = [evaluate_genome(genome, base_features, u_t, epsilon=0)[0] for genome in chrom]
# epi = 0; epi = 10**(sci_format(np.median(fitnesses))[1]-1+epi); print('epi =', epi)
# for i in range(len(fitnesses)):
#     fitnesses[i] += epi*len(chrom[i])

# print("Learning PDEs...")
# for _ in trange(n_generations):
#     chrom_f, chrom_m = train_test_split(chrom, test_size=0.5)
#     children = []
#     children_fitnesses = []
#     for genome_f, genome_m in zip(chrom_f, chrom_m):
#         genome_child1, genome_child2 = crossover(list(genome_f), list(genome_m))

#         if genome_child1 not in set_chrom:
#             fitness_child1, _ = evaluate_genome(genome_child1, base_features, u_t, epsilon=epi)
#             children.append(genome_child1)
#             children_fitnesses.append(fitness_child1)
#             set_chrom.add(genome_child1)
#         if genome_child2 not in set_chrom:
#             fitness_child2, _ = evaluate_genome(genome_child2, base_features, u_t, epsilon=epi)
#             children.append(genome_child2)
#             children_fitnesses.append(fitness_child2)
#             set_chrom.add(genome_child2)

#     chrom_fitnesses = heapq.nsmallest(len(chrom), zip(chrom+children, fitnesses+children_fitnesses), 
#                                       key=lambda _: _[1])
#     chrom = []
#     fitnesses = []
#     for genome, fitness in chrom_fitnesses:
#         chrom.append(genome); fitnesses.append(fitness)
    
# assert len(chrom) == len(set(chrom))
# aaa = chrom_fitnesses
# chrom_fitnesses[:10]

## GA ##

In [8]:
n_generations = 20
mutate_rate = 0.4

print("Initialization...")
chrom = generate_chromosome(n_modules, n_derivatives, pop_size)
set_chrom = chrom.copy()
chrom = list(chrom)

fitnesses = [evaluate_genome(genome, base_features, u_t, epsilon=0)[0] for genome in chrom]
epi = 0; epi = 10**(sci_format(np.median(fitnesses))[1]-1+epi); print('epi =', epi)
for i in range(len(fitnesses)):
    fitnesses[i] += epi*len(chrom[i])
    
best_chrom_fitnesses = heapq.nsmallest(len(chrom), zip(chrom, fitnesses), 
                                       key=lambda _: _[1])

print("Learning PDEs...")
for _ in trange(n_generations):
    chrom_f, chrom_m = train_test_split(chrom, test_size=0.5)
    for genome_f, genome_m in zip(chrom_f, chrom_m):
        genome_child1, genome_child2 = crossover(list(genome_f), list(genome_m))
        chrom.append(genome_child1)
        chrom.append(genome_child2)
        if genome_child1 not in set_chrom:
            fitness_child1, _ = evaluate_genome(genome_child1, base_features, u_t, epsilon=epi)
            best_chrom_fitnesses.append((genome_child1, fitness_child1))
            set_chrom.add(genome_child1)
        if genome_child2 not in set_chrom:
            fitness_child2, _ = evaluate_genome(genome_child2, base_features, u_t, epsilon=epi)
            best_chrom_fitnesses.append((genome_child2, fitness_child2))
            set_chrom.add(genome_child2)

    for i in range(len(chrom)):
        genome = list(chrom[i].copy())
        genome = mutation(genome, n_derivatives, mutate_rate)
        if len(genome) > 0:
            chrom[i] = fset(genome)
    
    new_chrom = []; fitnesses = []
    for i in range(len(chrom)):
        if chrom[i] not in set_chrom:
            fitness, _ = evaluate_genome(chrom[i], base_features, u_t, epsilon=epi)
            best_chrom_fitnesses.append((chrom[i], fitness))
            new_chrom.append(chrom[i])
            fitnesses.append(fitness)
            set_chrom.add(chrom[i])
            
    best_chrom_fitnesses = heapq.nsmallest(pop_size, best_chrom_fitnesses, key=lambda _ : _[1])
    chrom_fitnesses = heapq.nsmallest(pop_size, zip(new_chrom, fitnesses), key=lambda _ : _[1])
    chrom = []; fitnesses = []; del new_chrom
    for genome, fitness in chrom_fitnesses:
        chrom.append(genome); fitnesses.append(fitness)

assert len(chrom) == len(set(chrom))
bbb = best_chrom_fitnesses
best_chrom_fitnesses[:10]

Initialization...
epi = 1e-05
Learning PDEs...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.51it/s]


[(frozenset({(0, 1), (2,)}), 2.050332746595255e-05),
 (frozenset({(0, 1), (1, 1, 2), (2,)}), 3.041491578636353e-05),
 (frozenset({(0, 1), (1, 2), (2,)}), 3.0415188414112294e-05),
 (frozenset({(0, 1), (0, 1, 2), (2,)}), 3.0437682829447777e-05),
 (frozenset({(0, 0, 1), (0, 1), (2,)}), 3.0480306430264856e-05),
 (frozenset({(0, 0, 0), (0, 1), (2,)}), 3.048083262500738e-05),
 (frozenset({(0, 0), (0, 1), (2,)}), 3.048695770002536e-05),
 (frozenset({(0, 1), (1,), (2,)}), 3.0488302731385912e-05),
 (frozenset({(0, 1), (2,), (2, 2, 2)}), 3.049153258449998e-05),
 (frozenset({(0, 1), (1, 1, 1), (2,)}), 3.0493456169225435e-05)]

In [9]:
n_common = 10
common_genomes = []
for genome, _ in best_chrom_fitnesses:
    common_genomes.extend([*genome])
common_genomes = Counter(common_genomes).most_common(n_common)
common_genomes = fset(genome for genome, _ in common_genomes)
X_pre = numericalize_genome(common_genomes, base_features).reshape(-1, n_common)
y_pre = u_t.reshape(-1, 1)